<center><h1>Procesamiento y transformación de variables (características)</h1></center>

## 1. Introducción.

Para entender cómo funciona la regresión lineal, nos limitamos a usar características del conjunto de datos de entrenamiento que no contenían valores faltantes y que ya estaban en una representación numérica conveniente. En esta lección, exploraremos cómo transformar algunas de las características restantes para poder usarlas en nuestro modelo. En términos generales, el proceso de procesamiento y creación de nuevas características se conoce como ingeniería de características (**feature engineering**). La ingeniería de características o variables es un poco un arte y tener conocimiento en el dominio específico (en este caso, bienes raíces) puede ayudarlo a crear mejores características. En esta lección, nos centraremos en algunas estrategias independientes del dominio que funcionan para todos los problemas.

En la primera mitad de esta lección, nos centraremos solo en las columnas que no contienen valores perdidos pero que aún no tienen el formato adecuado para usar en un modelo de regresión lineal. En la segunda mitad de esta lección, exploraremos algunas formas de lidiar con los valores faltantes.

Entre las columnas que no contienen valores faltantes, algunos de los problemas comunes incluyen:

- la columna no es numérica (por ejemplo, un código de zonificación representado mediante texto)
- la columna es numérica pero no ordinal (por ejemplo, valores de código postal)
- la columna es numérica pero no es representativa del tipo de relación con la columna de destino (por ejemplo, valores de año)

Comencemos por filtrar el conjunto de entrenamiento a solo las columnas que no contienen valores faltantes.

### Ejercicio
- Seleccione solo las columnas del dataframe de `train` que no contengan valores faltantes.
- Asigne el marco de datos resultante, que contiene solo estas columnas, a `df_no_mv`.
- Explore las variables para familiarizarse con estas columnas.

## 2. Variables categóricas.
Notará que algunas de las columnas en el marco de datos `df_no_mv` contienen valores de cadena. Si estas columnas contienen solo un conjunto limitado de valores únicos, se conocen como  variables(características) categóricas. Como sugiere el nombre, una característica categórica agrupa una instancia en una categoría específica. Estos son algunos ejemplos del conjunto de datos:

Para usar estas características en nuestro modelo, necesitamos transformarlas en representaciones numéricas. Afortunadamente, pandas lo hace fácil porque la biblioteca tiene un tipo de [datos categórico](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html). Podemos convertir cualquier columna que no contenga valores faltantes (o se generará un error) al tipo de datos categóricos usando el método [`pandas.Series.astype()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html):

Cuando una columna se convierte a tipo de datos categóricos, pandas asigna un código a cada valor único en la columna. A menos que accedamos a estos valores directamente, la mayoría de las operaciones de manipulación de pandas que funcionan para las columnas de cadenas (texto) también funcionarán para las categóricas.

Necesitamos usar el descriptor de acceso `.cat` seguido de la propiedad `.codes` para acceder realmente a la representación numérica subyacente de una columna:

### Ejercicio
- Convierta todas las columnas de texto en `train` al tipo de datos categóricos. Para seleccionar columnas de un tipo de dato en específico puede usar [`pandas.DataFrame.select_dtypes`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html)
- Seleccione la columna `Utilities`, devuelva los códigos categóricos y muestre los recuentos de valores únicos para esos códigos: `train['Utilities'].cat.codes.value_counts()` 

## 3. Codificación ficticia.
Cuando convertimos una columna datos categóricos, pandas asigna un número `0` a `n-1` (donde `n` es el número de valores únicos en una columna) para cada valor. El inconveniente de este enfoque es que aquí se viola uno de los supuestos de la regresión lineal. La regresión lineal opera bajo el supuesto de que las características están linealmente correlacionadas con la columna de destino. Sin embargo, para una característica categórica, no hay un significado numérico real para los códigos categóricos que pandas asignó para esa columna. Un aumento en la columna Utilidades de 1 a 2 no tiene un valor de correlación con la columna de destino y, en cambio, los códigos categóricos se utilizan para la unicidad y exclusividad (la categoría asociada con 0 es diferente a la asociada con 1).

La solución común es usar una técnica llamada [codificación ficticia](https://en.wikipedia.org/wiki/Dummy_variable_%28statistics%29). En lugar de tener una sola columna con `n` códigos enteros, tenemos `n` columnas binarias. Así es como se vería para la columna Utilidades:

| Utilities_AllPub | Utilities_NoSewr | Utilities_NoSeWa |
|------------------|------------------|------------------|
| 1                | 0                | 0                |
| 1                | 0                | 0                |
| 1                | 0                | 0                |
| 1                | 0                | 0                |

Debido a que los valores originales para las primeras 4 filas eran `AllPub`, en el nuevo esquema contienen el valor binario para verdadero (`1`) en la columna `Utilities_AllPub` y `0` para las otras 2 columnas.

Pandas tiene una función conveniente para ayudarnos a aplicar esta transformación para todas las columnas de texto llamada [`pandas.get_dummies()`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).


### Ejercicio
- Convierta todas las columnas en `text_cols` del datafrane `train` en columnas ficticias.
- Elimine las columnas originales de `text_cols` del dataframe `train`.

## 4. Transformación de variables numéricas impropias.

Nos enfocamos en valores categóricos que se representaron como columnas de texto. Algunas de las columnas numéricas del conjunto de datos también son categóricas y solo tienen un conjunto limitado de valores únicos. No exploraremos explícitamente esas columnas por el momento, pero el proceso de transformación de características es el mismo si los números usados en esas categorías no tienen significado numérico.

Veamos ahora las características numéricas que no son categóricas, pero cuya representación numérica debe mejorarse. Nos centraremos en las columnas `Year Remod/Add` y `Year Built`:

Los dos problemas principales con estas características son:

- Los valores del año no son representativos de la antigüedad de una casa
- La columna Year Remod/Add en realidad no proporciona información útil para un modelo de regresión lineal

El desafío con valores de año como `1960` y `1961` es que no hacen un buen trabajo al capturar la antigüedad de una casa. Por ejemplo, una casa que se construyó en `1960` pero se vendió en `1980` se vendió en la mitad del tiempo que una construida en `1960` y vendida en `2000`. En lugar de los años en que ocurrieron ciertos eventos, queremos la diferencia entre esos años. Deberíamos crear una nueva columna que sea la diferencia entre ambas columnas.

Para esta información en particular (años hasta que se remodeló), este es un enfoque sensato. El conocimiento del negocio puede ayudar a comprender cómo transformar mejor las características para representar bien la información para un modelo lineal.

### Ejercicio
- Cree una nueva columna `years_until_remod` en el dataframe de `train` que represente la diferencia entre `Year Remod/Add` (el valor posterior) y `Year Built` (el valor anterior).

## 5. Valores faltantes.
A continuación nos centraremos en el manejo de columnas con valores faltantes. Cuando faltan valores en una columna, hay dos enfoques principales que podemos tomar:

- Eliminar filas que contienen valores faltantes para columnas específicas
    - Ventaja: las filas que contienen valores faltantes se eliminan, dejando solo datos limpios para el modelado
    - Desventaja: Se eliminan todas las observaciones del conjunto de entrenamiento, lo que puede reducir la precisión general de la predicción.
- Impute (o reemplace) los valores faltantes usando una estadística descriptiva de la columna
    - Ventaja: los valores faltantes se reemplazan con estimaciones potencialmente similares, conservando el resto de la observación en el modelo.
    - Desventaja: dependiendo del enfoque, es posible que estemos agregando datos ruidosos para que el modelo los aprenda.

Dado que solo tenemos 1460 ejemplos de entrenamiento (con ~80 variables potencialmente útiles), no queremos eliminar ninguna de estas filas del conjunto de datos. En cambio, centrémonos en las técnicas de **imputación**.

Nos centraremos en las columnas que contienen entre 1  y 365 valores faltantes (o el 25 % del número de filas en el conjunto de entrenamiento). No existe un umbral estricto y, en cambio, muchas veces se usa un límite del 50 % (si falta la mitad de los valores en una columna, se descarta automáticamente). Tener conocimiento del negocio puede ayudar a determinar un valor de corte aceptable.

### Ejercicio

- Seleccione solo las columnas de `train` que contengan más de `0` pero menos de `584` valores perdidos. Asigne el dataframe resultante a `df_missing_values`.
- Muestra el número de valores que faltan para cada columna en `df_missing_values`.
- Muestre el tipo de datos para cada columna en `df_missing_values`.

## 6. Imputación de valores faltantes.
Parece que aproximadamente la mitad de las columnas en `df_missing_values` son columnas de cadena (datos tipo `object`), mientras que aproximadamente la mitad son columnas `float64`. Para columnas numéricas con valores faltantes, una estrategia común es calcular la media, la mediana o la moda de cada columna y reemplazar todos los valores faltantes en esa columna con ese valor.

Debido a que la imputación es una tarea común, pandas contiene un método [`pandas.DataFrame.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) que podemos usar para esto. Si pasamos un valor, todos los valores faltantes (`NaN`) en el dataframe se reemplazan por ese valor:

```python
# Only select float columns.
missing_floats = df_missing_vals.select_dtypes(include=['float'])

# Returns a data frame with missing values replaced with 0.
fill_with_zero = missing_floats.fillna(0)
```

También puede pasar una función por columnas y completar los valores faltantes de esa manera:

```python
# Returns a data frame with missing values replaced with mean of that column.
fill_with_mean = missing_floats.fillna(missing_floats.mean())
```

### Ejercicio
- Impute los valores faltantes de `float_cols` con la media de la columna.
- Compruebe si faltan valores en `float_cols`.

## 7. Conclusión.
En esta lección, exploramos diferentes técnicas  para transformar entidades en representaciones apropiadas para un modelo de regresión lineal.